# 006_langgraph_tutorial_1

## `01` Import Libraries

In [87]:
from dotenv import load_dotenv
import os

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama


from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

from typing import Sequence, List

## `02` Load API Keys

In [77]:
load_dotenv(".env")

True

In [78]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = os.getenv("GEMINI_MODEL")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "false")

## `03` Prompts temp

In [79]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a virtual twitter innfluencer grading a tweet. Generate critique and suggestions for the user."
            "Always provide a score from 1 to 10, with 10 being the best."
            "Always provide deailed recommendations, including requests for length, hashtags, and emojis.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [80]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

## `04` LLM

In [ ]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",
#     max_output_tokens=100,
#     api_key= GEMINI_API_KEY,
#     temperature=0.2,
# )

llm = ChatOllama( ## I use llama3 when my api key reaches the limit
    model="llama3",
)

Flows

In [ ]:
generate_chain = generation_prompt | llm 
reflect_chain = reflection_prompt | llm

In [90]:
def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT

Naming the nodes

In [91]:
REFLECT = "reflect"
GENERATE = "generate"

Build the graph

In [92]:
# Step 3: Build the graph
builder = MessageGraph()
builder.add_node("GENERATE", generation_node)
builder.add_node("REFLECT", reflection_node)
builder.set_entry_point("GENERATE")

builder.add_conditional_edges(
    "GENERATE", should_continue, {"reflect": "REFLECT", "__end__": "__end__"}
)

builder.add_edge("REFLECT", "GENERATE")

# Step 4: Compile and visualize
graph = builder.compile()
print(graph.get_graph().draw_mermaid())
graph.get_graph().print_ascii()

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	GENERATE(GENERATE)
	REFLECT(REFLECT)
	__end__([<p>__end__</p>]):::last
	GENERATE -. &nbsp;reflect&nbsp; .-> REFLECT;
	GENERATE -.-> __end__;
	REFLECT --> GENERATE;
	__start__ --> GENERATE;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

          +-----------+            
          | __start__ |            
          +-----------+            
                *                  
                *                  
                *                  
          +----------+             
          | GENERATE |             
          +----------+             
          ***        ...           
         *              .          
       **                ..        
+---------+           +---------+  
| REFLECT |           | __end__ |  
+---------+           +---------+  


In [93]:
print("Hello LangGraph")
inputs = HumanMessage(
    content="""Make this tweet better:"
                                @LangChainAI
        — newly Tool Calling feature is seriously underrated.

        After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

        Made a video covering their newest blog post

                                """
)
response = graph.invoke(inputs)

Hello LangGraph


In [94]:
print(response)

[HumanMessage(content='Make this tweet better:"\n                                @LangChainAI\n        — newly Tool Calling feature is seriously underrated.\n\n        After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n        Made a video covering their newest blog post\n\n                                ', additional_kwargs={}, response_metadata={}, id='7d8b7606-e8d9-4bbb-b906-a06982cf71c7'), AIMessage(content='Here\'s a revised version of your tweet:\n\n"Get ready to level up your AI game! The newly released @LangChainAI Tool Calling feature is a total game-changer. Say goodbye to complexity and hello to seamless integration of agents across different models with function calling. Watch my latest video covering the new blog post and discover how you can simplify your workflow: [link] #AILang #ToolCalling"\n\nI made some changes to make the tweet more engaging:\n\n* Started with a hook that grabs attentio